In [2]:
import ccxt.async_support as ccxt
import pandas as pd
import asyncio
import os
import sys
from pprint import pprint

In [3]:
df = pd.read_pickle("binanceus_DOGE-USDT_1620688812541_1620688872541_lob.pkl")

In [7]:
df

,date,exchange_id,symbol,asks,bids
0,2021-05-10T23:20:13.085Z,binanceus,DOGE/USDT,"[[0.4479014, 476.0], [0.4479726, 45.0], [0.448...","[[0.4475, 100.0], [0.4474383, 224.0], [0.44731..."
1,2021-05-10T23:20:13.593Z,binanceus,DOGE/USDT,"[[0.4477604, 399.0], [0.4479014, 1.0], [0.4479...","[[0.4475796, 292.0], [0.4475, 100.0], [0.44743..."
2,2021-05-10T23:20:14.090Z,binanceus,DOGE/USDT,"[[0.4477604, 399.0], [0.4478105, 4464.0], [0.4...","[[0.4475796, 292.0], [0.4475, 100.0], [0.44743..."
3,2021-05-10T23:20:14.591Z,binanceus,DOGE/USDT,"[[0.4477604, 399.0], [0.4478105, 4464.0], [0.4...","[[0.4471995, 4250.0], [0.4470597, 352.0], [0.4..."
4,2021-05-10T23:20:15.166Z,binanceus,DOGE/USDT,"[[0.4476204, 374.0], [0.4477604, 399.0], [0.44...","[[0.4473696, 296.0], [0.4471995, 4250.0], [0.4..."
...,...,...,...,...,...
115,2021-05-10T23:21:10.596Z,binanceus,DOGE/USDT,"[[0.4502588, 518.0], [0.4503693, 1307.0], [0.4...","[[0.4499839, 369.0], [0.4494738, 8235.0], [0.4..."
116,2021-05-10T23:21:11.125Z,binanceus,DOGE/USDT,"[[0.4502588, 518.0], [0.4503693, 1307.0], [0.4...","[[0.4494992, 1084.0], [0.4493381, 8781.0], [0...."
117,2021-05-10T23:21:11.663Z,binanceus,DOGE/USDT,"[[0.4503693, 1307.0], [0.4504798, 1960.0], [0....","[[0.4496608, 868.0], [0.4494992, 1084.0], [0.4..."
118,2021-05-10T23:21:12.090Z,binanceus,DOGE/USDT,"[[0.4503693, 1307.0], [0.4504798, 1960.0], [0....","[[0.4496992, 8133.0], [0.4496608, 868.0], [0.4..."
